In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [3]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
744,(2021/05/31) 23-24,41.11,96
745,Min,26.98,17
746,Max,299.66,286
747,Average,83.52659946236559139784946237,106.83252688172043010752688172


In [4]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
740,(2021/05/31) 19-20,288.09,53.8
741,(2021/05/31) 20-21,118.04,60.0
742,(2021/05/31) 21-22,79.60,73.0
743,(2021/05/31) 22-23,41.15,126.0


In [5]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
740,(2021/05/31) 19-20,288.09,53.8,2021-05-31,19
741,(2021/05/31) 20-21,118.04,60.0,2021-05-31,20
742,(2021/05/31) 21-22,79.60,73.0,2021-05-31,21
743,(2021/05/31) 22-23,41.15,126.0,2021-05-31,22


In [6]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
740,288.09,53.8,2021-05-31,19
741,118.04,60.0,2021-05-31,20
742,79.60,73.0,2021-05-31,21
743,41.15,126.0,2021-05-31,22


In [7]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2021-05-31,288.09,53.8,19,2021,5,31,0
2021-05-31,118.04,60.0,20,2021,5,31,0
2021-05-31,79.60,73.0,21,2021,5,31,0


In [8]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    7319.5          120  345982.021        47.268532     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 5     27   2247.5           72  195365.359        86.925632     299.05   
           28   1778.9           96  169945.495        95.534035     298.83   
           29   3695.1          100  146378.094        39.614109      64.96   
           30   3280.0          120  113233.630        34.522448      40.08   
           31   1684.6            0  163582.794        97.104828     298.06   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2016 1     1        35.09   45.982500      502.9       53.0  328.075000   
           2        40.03   54.114167      486.0       40.0  304.979167   
           3        44.06   54.959167      486.0       40.0  299.945833   
           4        46.70   80.164583      486.0        0.0  185.708333   
           5        42.06   84.503333      436.0        0.0  156.266667   
...                   ...         ...        ...        ...         ...   
2021 5     27       36.09   98.355833      132.0       52.2   93.645833   
           28       36.03   97.538750      194.0       38.0   74.120833   
           29       27.18   41.100500      246.0      106.0  184.755000   
           30       27.18   35.110000      286.0      102.0  164.000000   
           31       36.12  109.443000      198.0       53.8   84.230000   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2016 1     1                48.03              60.08         45.8275  
           2                56.05             118.49         50.2275  
           3                56.01             118.47         50.6050  
           4                80.02             177.66         80.0200  
           5               148.40             181.40         80.0500  
...                           ...                ...             ...  
2021 5     27              167.95             299.05         77.9050  
           28              167.67             298.83         77.0425  
           29               45.44              64.96         36.2825  
           30               35.56              40.08         35.5175  
           31              167.57             298.06         77.0650  

[1978 rows x 13 columns]

In [9]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2021-05-31,288.09,53.8,19,2021,5,31,0,15499.242,evening
2021-05-31,118.04,60.0,20,2021,5,31,0,7082.400,standard
2021-05-31,79.60,73.0,21,2021,5,31,0,5810.800,standard


In [10]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    7319.5          120  345982.021        47.268532     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 5     27   2247.5           72  195365.359        86.925632     299.05   
           28   1778.9           96  169945.495        95.534035     298.83   
           29   3695.1          100  146378.094        39.614109      64.96   
           30   3280.0          120  113233.630        34.522448      40.08   
           31   1684.6            0  163582.794        97.104828     298.06   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2016 1     1        35.09   45.982500      502.9       53.0  328.075000  ...   
           2        40.03   54.114167      486.0       40.0  304.979167  ...   
           3        44.06   54.959167      486.0       40.0  299.945833  ...   
           4        46.70   80.164583      486.0        0.0  185.708333  ...   
           5        42.06   84.503333      436.0        0.0  156.266667  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 5     27       36.09   98.355833      132.0       52.2   93.645833  ...   
           28       36.03   97.538750      194.0       38.0   74.120833  ...   
           29       27.18   41.100500      246.0      106.0  184.755000  ...   
           30       27.18   35.110000      286.0      102.0  164.000000  ...   
           31       36.12  109.443000      198.0       53.8   84.230000  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      5969.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2021 5     27                  3.0                 2.0       917.0   
           28                  3.0                 2.0       458.0   
           29                  NaN                 NaN      2623.0   
           30                  NaN                 NaN      3280.0   
           31                  3.0                 2.0       402.0   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  271567.021   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2021 5     27             969.3           249.0           112.2   36352.780   
           28             979.0           228.0           113.9   17277.250   
           29            1072.1             NaN             NaN   93158.670   
           30               NaN             NaN             NaN  113233.630   
           31             946.0           222.8           113.8   15955.460   

        

In [11]:
daily.to_csv(out + pref + '_daily.csv')

In [12]:
hourly.to_csv(out + pref + '_hourly.csv')

In [13]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat.loc[(hourly_flat['year']<2021)][['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2020-12-31,19,160.03,60.0,evening
2020-12-31,20,60.06,40.0,standard
2020-12-31,21,60.08,64.0,standard


In [14]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [15]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3785.351613,74.709677,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,...,42.0,55.986949,50.402420,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000
1,2016,2,3871.755172,69.517241,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,...,42.0,67.432548,56.605271,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952
2,2016,3,2904.254839,68.903226,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,...,46.0,64.633890,55.295628,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957
3,2016,4,1921.116667,73.200000,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,...,42.0,61.925094,57.116312,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095
4,2016,5,1597.551613,69.548387,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,...,44.0,56.288962,49.684509,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021,1,2451.522581,74.709677,148903.379194,88.205417,44.004000,64.075785,184.783871,42.096774,...,42.0,60.739142,49.217227,65.490663,96.988945,117.813297,112.052355,84.269841,104.337218,59.133333
61,2021,2,2225.446429,72.000000,183252.046929,136.436250,43.510417,84.913795,141.325000,48.175000,...,40.0,82.343949,56.300449,96.540477,124.672487,160.354642,92.895988,97.925000,96.015726,63.170000
62,2021,3,2729.619355,66.967742,157902.258484,78.673750,27.023333,61.538008,189.854839,58.119355,...,46.0,57.847721,40.743795,61.365324,107.069933,131.375021,120.049405,87.672464,122.960142,80.008696
63,2021,4,2688.473333,71.866667,161123.687933,110.314000,30.873750,80.865714,185.530000,40.316667,...,44.0,59.931295,37.210893,66.760284,142.181426,252.474165,132.883133,74.968182,109.424815,46.463636


In [16]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3785.351613,74.709677,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,...,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3871.755172,69.517241,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,...,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2904.254839,68.903226,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,...,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1921.116667,73.200000,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,...,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1597.551613,69.548387,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,...,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2021,1,2451.522581,74.709677,148903.379194,88.205417,44.004000,64.075785,184.783871,42.096774,...,65.490663,96.988945,117.813297,112.052355,84.269841,104.337218,59.133333,21,21,31
61,2021,2,2225.446429,72.000000,183252.046929,136.436250,43.510417,84.913795,141.325000,48.175000,...,96.540477,124.672487,160.354642,92.895988,97.925000,96.015726,63.170000,20,20,28
62,2021,3,2729.619355,66.967742,157902.258484,78.673750,27.023333,61.538008,189.854839,58.119355,...,61.365324,107.069933,131.375021,120.049405,87.672464,122.960142,80.008696,23,23,31
63,2021,4,2688.473333,71.866667,161123.687933,110.314000,30.873750,80.865714,185.530000,40.316667,...,66.760284,142.181426,252.474165,132.883133,74.968182,109.424815,46.463636,22,22,30


In [17]:
monthly.to_csv(out + pref + '_monthly.csv')

In [18]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [19]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,off_price_median,standard_price_median,morning_price_median,evening_price_median,volume,off_volume,standard_volume,morning_volume,evening_volume,value
0,1,70.032258,173118.297121,46.277157,97.634677,33.015968,57.730372,302.916129,46.225000,163.875013,...,32.163750,68.129091,69.963333,86.220000,120165.475,84276.100,27816.775,5167.975,2904.625,5.366667e+06
1,2,71.362832,180944.057319,49.294629,103.178584,32.019646,59.592395,290.303540,39.270796,153.693798,...,30.950000,69.182727,66.326667,84.425000,104087.700,69493.075,26354.975,5379.275,2860.375,5.111670e+06
2,3,71.806452,123187.500298,46.408978,98.810161,29.081210,54.249944,206.841935,41.371774,109.563219,...,25.801875,61.694545,63.903333,82.185000,80257.875,47453.900,23659.500,6207.825,2936.650,3.818813e+06
3,4,71.700000,147987.117217,47.915887,97.065000,27.734167,54.287413,211.321667,53.966667,127.345444,...,27.533125,65.030000,60.621667,97.945000,90737.875,48786.100,29595.050,8184.400,4172.325,4.439614e+06
4,5,69.064516,122776.968742,47.193876,111.561774,29.676290,58.420473,206.340323,36.551613,104.580907,...,32.216875,68.496364,74.916667,101.055000,76563.400,48241.200,20351.700,5538.400,2432.100,3.806086e+06
5,6,72.533333,215518.876867,59.910024,135.855750,36.991667,73.653660,293.115833,43.570000,146.700597,...,37.350000,77.056000,124.450000,130.693333,104400.975,68721.775,23696.300,6554.725,5428.175,6.465566e+06
6,7,70.838710,289812.474097,65.678159,145.048952,40.198548,79.022042,343.320968,56.512903,182.036028,...,41.927500,97.203750,137.826667,141.870000,133006.775,84364.425,32715.200,8958.050,6969.100,8.984187e+06
7,8,69.870968,319384.388218,63.357590,146.385161,37.325000,78.214727,404.586290,67.423387,213.756801,...,33.181875,89.392000,134.956667,140.216667,156751.600,98366.550,40083.425,9914.150,8387.475,9.900916e+06
8,9,72.700000,328725.183258,52.947464,119.131000,33.521083,64.200813,462.670833,107.923333,262.206167,...,31.895000,64.929091,98.431667,152.372500,186950.475,118249.125,51069.725,11650.225,5981.400,9.861755e+06
9,10,69.387097,351936.850653,49.757968,105.153306,31.887339,56.908052,494.074194,116.862097,305.467560,...,31.305000,65.024481,85.170000,112.185000,223533.900,135823.925,69330.775,13605.925,4773.275,1.091004e+07


In [20]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [21]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      38.758631  235.642857   9003.812315
      1      38.022976  236.169643   8951.946917
      2      37.682440  235.802381   8904.334869
      3      37.581905  238.243452   8995.861917
      4      37.673172  238.518280   9002.655156
...                ...         ...           ...
12    19    130.661613   80.230968   8684.619781
      20     89.013548  105.837419   7205.543774
      21     73.281871  171.339355  10649.464465
      22     39.710839  254.710968   8996.688987
      23     37.374000  260.658710   9130.851213

[288 rows x 3 columns]

In [182]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')